In [1]:
# Importing required libraries and dataset

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

train_df = pd.read_csv("../input/envision-racing/train.csv")
test_df = pd.read_csv("../input/envision-racing/test.csv")
train_weather_df = pd.read_csv("../input/envision-racing/train_weather.csv")
test_weather_df = pd.read_csv("../input/envision-racing/test_weather.csv")

FileNotFoundError: ignored

In [ ]:
# Exploratory Data Analysis

train_df.info()

# Observations:
# There are 10,276 rows and 25 columns
# Null values are present in some columns
# Time columns are of string data type (need to convert it to numeric)

In [ ]:
test_df.info()

# Observations:
# There are 420 rows and 25 columns
# Null values are present in some columns
# Time columns are of string data type (need to convert it to numeric)

In [ ]:
#Removing leading and trailing spaces from column names

train_df.columns = list(pd.Series(train_df.columns).apply(lambda x:x.strip()))
test_df.columns = list(pd.Series(test_df.columns).apply(lambda x:x.strip()))

In [ ]:
# Creating a function to convert time to seconds

def time_to_sec(time):
    if type(time) != str:
        return time
    
    parts = [float(p) for p in time.split(':')]
    parts_sec = [p * (60**i) for i,p in enumerate(reversed(parts))] 
    return sum(parts_sec)

In [ ]:
# Converting all time columns into numeric (seconds)

time_columns = ['S1','S2','S3','ELAPSED','HOUR','S1_LARGE','S2_LARGE','S3_LARGE','PIT_TIME']

for col in time_columns:
    train_df[col] = train_df[col].apply(time_to_sec)
    test_df[col] = test_df[col].apply(time_to_sec)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
# Observations:
# Train weather file has information regarding locations 1 to 7 and test weather file has information
# regarding location 8
# Train weather file has values in the acceptable range of values for locations 1,2,3,4 
# Replace , with . and convert the data to numeric

train_weather_df1 = train_weather_df[train_weather_df['LOCATION'].isin(['Location 1','Location 2','Location 3','Location 4'])]

train_weather_df1['AIR_TEMP'] = train_weather_df['AIR_TEMP'].str.replace(',','.')
train_weather_df1['AIR_TEMP'] = pd.to_numeric(train_weather_df1['AIR_TEMP'])
train_weather_df1['TRACK_TEMP'] = train_weather_df['TRACK_TEMP'].str.replace(',','.')
train_weather_df1['TRACK_TEMP'] = pd.to_numeric(train_weather_df1['TRACK_TEMP'])
train_weather_df1['HUMIDITY'] = train_weather_df['HUMIDITY'].str.replace(',','.')
train_weather_df1['HUMIDITY'] = pd.to_numeric(train_weather_df1['HUMIDITY'])
train_weather_df1['PRESSURE'] = train_weather_df['PRESSURE'].str.replace(',','.')
train_weather_df1['PRESSURE'] = pd.to_numeric(train_weather_df1['PRESSURE'])
train_weather_df1['WIND_SPEED'] = train_weather_df['WIND_SPEED'].str.replace(',','.')
train_weather_df1['WIND_SPEED'] = pd.to_numeric(train_weather_df1['WIND_SPEED'])
train_weather_df1['RAIN'] = train_weather_df['RAIN'].str.replace(',','.')
train_weather_df1['RAIN'] = pd.to_numeric(train_weather_df1['RAIN'])

In [ ]:
# Observations:
# Train weather file has unacceptable range of values for locations 5,6,7 
# Rescaling the values and replace , with . and convert the data to numeric

train_weather_df2 = train_weather_df[train_weather_df['LOCATION'].isin(['Location 5','Location 6','Location 7'])]

train_weather_df2['AIR_TEMP'] = train_weather_df['AIR_TEMP'].str.replace(',','')
train_weather_df2['AIR_TEMP'] = pd.to_numeric(train_weather_df1['AIR_TEMP'])

conditions = [
    (train_weather_df2['AIR_TEMP'] > 100) & (train_weather_df2['AIR_TEMP'] <= 1000),
    (train_weather_df2['AIR_TEMP'] > 1000) & (train_weather_df2['AIR_TEMP'] <= 10000),
    (train_weather_df2['AIR_TEMP'] > 10000) & (train_weather_df2['AIR_TEMP'] <= 100000),
    (train_weather_df2['AIR_TEMP'] > 100000)]

choices = [train_weather_df2['AIR_TEMP']/10,train_weather_df2['AIR_TEMP']/100,
           train_weather_df2['AIR_TEMP']/1000,train_weather_df2['AIR_TEMP']/10000]

train_weather_df2['AIR_TEMP'] = np.select(conditions, choices)

train_weather_df2['TRACK_TEMP'] = train_weather_df['TRACK_TEMP'].str.replace(',','.')
train_weather_df2['TRACK_TEMP'] = pd.to_numeric(train_weather_df2['TRACK_TEMP'])

train_weather_df2['HUMIDITY'] = train_weather_df['HUMIDITY'].str.replace(',','.')
train_weather_df2['HUMIDITY'] = pd.to_numeric(train_weather_df2['HUMIDITY'])

train_weather_df2['PRESSURE'] = train_weather_df['PRESSURE'].str.replace(',','')
train_weather_df2['PRESSURE'] = pd.to_numeric(train_weather_df1['PRESSURE'])

conditions = [
    (train_weather_df2['PRESSURE'] > 10000) & (train_weather_df2['PRESSURE'] <= 20000),
    (train_weather_df2['PRESSURE'] > 20000) & (train_weather_df2['PRESSURE'] <= 200000),
    (train_weather_df2['PRESSURE'] > 200000)]

choices = [train_weather_df2['PRESSURE']/10,train_weather_df2['PRESSURE']/100,train_weather_df2['PRESSURE']/1000]

train_weather_df2['PRESSURE'] = np.select(conditions, choices)

train_weather_df2['WIND_SPEED'] = train_weather_df['WIND_SPEED'].str.replace(',','')
train_weather_df2['WIND_SPEED'] = pd.to_numeric(train_weather_df1['WIND_SPEED'])

conditions = [
    (train_weather_df2['WIND_SPEED'] > 10) & (train_weather_df2['WIND_SPEED'] <= 100),
    (train_weather_df2['WIND_SPEED'] > 100) & (train_weather_df2['WIND_SPEED'] <= 1000),
    (train_weather_df2['WIND_SPEED'] > 1000) & (train_weather_df2['WIND_SPEED'] <= 10000),
    (train_weather_df2['WIND_SPEED'] > 10000) & (train_weather_df2['WIND_SPEED'] <= 100000),
    (train_weather_df2['WIND_SPEED'] > 100000)]

choices = [train_weather_df2['WIND_SPEED']/10,train_weather_df2['WIND_SPEED']/100,
           train_weather_df2['WIND_SPEED']/1000,train_weather_df2['WIND_SPEED']/10000,
           train_weather_df2['WIND_SPEED']/100000]

train_weather_df2['WIND_SPEED'] = np.select(conditions, choices)

train_weather_df2['RAIN'] = train_weather_df['RAIN'].str.replace(',','.')
train_weather_df2['RAIN'] = train_weather_df2['RAIN'].astype('float')

In [ ]:
# concatenating modified train_weather data frames across locations 1 to 7

train_weather_modified_df = pd.concat([train_weather_df1,train_weather_df2])
train_weather_modified_df.info()

In [ ]:
# Performing same operations on test weather file

test_weather_df['AIR_TEMP'] = test_weather_df['AIR_TEMP'].str.replace(',','.')
test_weather_df['AIR_TEMP'] = pd.to_numeric(test_weather_df['AIR_TEMP'])
test_weather_df['TRACK_TEMP'] = test_weather_df['TRACK_TEMP'].str.replace(',','.')
test_weather_df['TRACK_TEMP'] = pd.to_numeric(test_weather_df['TRACK_TEMP'])
test_weather_df['HUMIDITY'] = test_weather_df['HUMIDITY'].str.replace(',','.')
test_weather_df['HUMIDITY'] = pd.to_numeric(test_weather_df['HUMIDITY'])
test_weather_df['PRESSURE'] = test_weather_df['PRESSURE'].str.replace(',','.')
test_weather_df['PRESSURE'] = pd.to_numeric(test_weather_df['PRESSURE'])
test_weather_df['WIND_SPEED'] = test_weather_df['WIND_SPEED'].str.replace(',','.')
test_weather_df['WIND_SPEED'] = pd.to_numeric(test_weather_df['WIND_SPEED'])

In [ ]:
# Renaming EVENTS column to EVENT in test dataset
test_weather_df.rename(columns={'EVENTS':'EVENT'},inplace=True)
test_weather_df.info()

In [ ]:
# Summarizing weather information across locations and events

train_weather_summary_df = train_weather_modified_df.groupby(['LOCATION','EVENT'])['AIR_TEMP','TRACK_TEMP','HUMIDITY','PRESSURE','WIND_SPEED','WIND_DIRECTION','RAIN'].mean().reset_index()
test_weather_summary_df = test_weather_df.groupby(['LOCATION','EVENT'])['AIR_TEMP','TRACK_TEMP','HUMIDITY','PRESSURE','WIND_SPEED','WIND_DIRECTION','RAIN'].mean().reset_index()

In [ ]:
# Merging train and test data with master weather data

train_df1 = train_df.merge(train_weather_summary_df, left_on=['LOCATION','EVENT'], right_on=['LOCATION','EVENT'], how='left')
test_df1 = test_df.merge(test_weather_summary_df, left_on=['LOCATION','EVENT'], right_on=['LOCATION','EVENT'], how='left')

In [ ]:
train_df1.info()

In [ ]:
test_df1.info()

In [ ]:
# Combining train and test data
train_test_df = pd.concat([train_df1,test_df1],axis=0)

# One-hot Encoding categorical columns - driver_name, team, location and event
train_test_df = pd.get_dummies(train_test_df)

# Separating train and test sets
train_df2 = train_test_df.iloc[:len(train_df)]
test_df2 = train_test_df.iloc[len(train_df):]

In [ ]:
train_df2.info()

In [ ]:
test_df2.info()

In [ ]:
print(train_df2.shape)
print(test_df2.shape)

In [ ]:
# Lisiting continuous columns from train and test data

cont_cols = ['S1','S2','S3','KPH','ELAPSED','HOUR','S1_LARGE','S2_LARGE','S3_LARGE','PIT_TIME',
             'AIR_TEMP','TRACK_TEMP','HUMIDITY','WIND_SPEED','WIND_DIRECTION']

In [ ]:
# Comparing train vs test data distribution

for col in cont_cols:
    
    plt.figure(figsize=(12,4))
    
    # plot train data
    plt.subplot(1, 2, 1)
    plt.hist(train_df2[col], bins=50)
    plt.xlabel('Train ' + col)

    # plot test data
    plt.subplot(1, 2, 2)
    plt.hist(test_df2[col], bins=50)
    plt.xlabel('Test ' + col)
                
    plt.show()
    
    # Observations:
    # Almost similar distributions for train and test data

In [ ]:
# Analyzing the distributions of the variables after applying yeo-johnson transformation

import scipy.stats as stats

# temporary copy of the data
train_yj_df2 = train_df2.copy()

for col in cont_cols:

    # transform the variable - yeo-johsnon
    train_yj_df2[col], param = stats.yeojohnson(train_df2[col])

    
# plot the histograms of the transformed variables
train_yj_df2[cont_cols].hist(bins=50, figsize=(20,20))
plt.show()

In [ ]:
# Analyzing the distributions of the variables after applying logarithmic transformation

# temporary copy of the data
train_log_df2 = train_df2.copy()

for col in cont_cols:

    # transform the variable - log
    train_log_df2[col] = np.log(train_df2[col]+1)

    
# plot the histograms of the transformed variables
train_log_df2[cont_cols].hist(bins=50, figsize=(20,20))
plt.show()

In [ ]:
# Plotting the original and transformed variables to see how the distribution varies

for col in cont_cols:
    
    plt.figure(figsize=(12,4))
    
    # plot the original variable  
    plt.subplot(1, 3, 1)
    plt.hist(train_df2[col], bins=50)
    plt.xlabel('Original ' + col)

    # plot yeo-johnson transformed variable
    plt.subplot(1, 3, 2)
    plt.hist(train_yj_df2[col], bins=50)
    plt.xlabel('Yeo-Johnson ' + col)
    
    # plot log transformed variable
    plt.subplot(1, 3, 3)
    plt.hist(train_log_df2[col], bins=50)
    plt.xlabel('Log ' + col)
                
    plt.show()

In [ ]:
# Plotting the feature variables vs LAP_TIME, and see if there is a relationship

for col in cont_cols:
    
    plt.figure(figsize=(20,4))
    
    # plot the original variable vs lap time   
    plt.subplot(1, 3, 1)
    plt.scatter(train_df2[col], train_df2['LAP_TIME'])
    plt.ylabel('LAP_TIME')
    plt.xlabel('Original ' + col)
    
    # plot Yeo-Johnson transformed variable vs lap time
    plt.subplot(1, 3, 2)
    plt.scatter(train_yj_df2[col], train_yj_df2['LAP_TIME'])
    plt.ylabel('LAP_TIME')
    plt.xlabel('Yeo-Johnson Transformed ' + col)

    # plot log-transformed variable vs lap time
    plt.subplot(1, 3, 3)
    plt.scatter(train_log_df2[col], train_log_df2['LAP_TIME'])
    plt.ylabel('LAP_TIME')
    plt.xlabel('Log Transformed ' + col)
                
    plt.show()
    
# Decisions:
# S1-Log, S2-yj, S3-yj, KPH-log, ELAPSED & HOUR-yj, S1_LARGE-log, S2_LARGE & S3_LARGE-yj
# PIT_TIME-log, Rest-Original

In [ ]:
#!pip install pycaret

# Note: After exploring different libraries like sci-kit learn and h2O, I have chosen PyCaret as 
# it was giving the best performance for this dataset

In [ ]:
from pycaret.regression import *
reg_models = setup(data = train_df2, target='LAP_TIME', session_id=123, normalize=True)

In [ ]:
model_comparison = compare_models()

In [ ]:
# Training the data on the best performing model --Orthogonal Matching Pursuit (OMP)

omp_model = create_model('omp')

In [ ]:
# Tuning the model

tuned_omp_model = tune_model(omp_model)

In [ ]:
# Predictions on Test data
predictions = predict_model(tuned_omp_model, data=test_df2)

# Exporting predictions to an excel file
pred_df = pd.DataFrame(predictions['Label'])
pred_df.rename(columns={'Label':'LAP_TIME'},inplace=True)
pred_df.to_csv("omp_pred2.csv",index=False)